# *`Nom & Prenom : `*

## Abal Khassim TRAORE

## Arona GUEYE

## Fatou CISSE

In [1]:
//creation d un dataframe
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`sh.almond::almond-spark:0.10.9`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`



import $ivy.$                                  

import $ivy.$                               

import $ivy.$                                    



In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql._


val spark = {
  SparkSession.builder()
    .master("local")
    .appName("BD-FS FIRE")
    .getOrCreate()
}



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._

import org.apache.spark.sql._



spark: SparkSession = org.apache.spark.sql.SparkSession@672020f8

# Importation du DataFrame

In [3]:
val path = "data/archive/*"
val df = spark
        .read
        .option("header","true")
        .csv(path)

path: String = "data/archive/*"
df: DataFrame = [Order ID: string, Product: string ... 4 more fields]

In [4]:
df.count()
df.columns
df.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



res3_0: Long = 186850L
res3_1: Array[String] = Array(
  "Order ID",
  "Product",
  "Quantity Ordered",
  "Price Each",
  "Order Date",
  "Purchase Address"
)

## Encodage de la colone "Order Date" en timestamp

In [5]:
import org.apache.spark.sql.functions.to_timestamp
import spark.implicits._


val dfWithTimestamp = df.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))


import org.apache.spark.sql.functions.to_timestamp

import spark.implicits._



dfWithTimestamp: DataFrame = [Order ID: string, Product: string ... 4 more fields]

In [6]:
import org.apache.spark.sql.functions.{to_timestamp, year}


val dfWithTimestamp = df.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))


import org.apache.spark.sql.functions.{to_timestamp, year}



dfWithTimestamp: DataFrame = [Order ID: string, Product: string ... 4 more fields]

## Q1: Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette année-là ?

In [7]:
// Chiffre d'affaire par mois trier par ordre descroissant
val dfAnnee = dfWithTimestamp
  .groupBy(year(col("Order Date")).as("Annee"))
  .agg(sum(col("Quantity Ordered") * col("Price Each")).as("Total"))
  .orderBy(desc("Total"))

  

// On prend le premier element de la colone
val bestAnnee = dfAnnee.first().getAs[Int]("Annee")
val total = dfAnnee.first().getAs[Double]("Total")

println(s"La meilleur annee en terme de vente est 20$bestAnnee avec un total de $total dollars.")

La meilleur annee en terme de vente est 2019 avec un total de 3.379903640992789E7 dollars.


dfAnnee: Dataset[Row] = [Annee: int, Total: double]
bestAnnee: Int = 19
total: Double = 3.379903640992789E7

## Q2: Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?

In [8]:
// Chiffre d'affaire par mois trier par ordre descroissant
val dfMois = dfWithTimestamp
  .groupBy(month(col("Order Date")).alias("Mois"))
  .agg(sum(col("Quantity Ordered") * col("Price Each")).as("Total"))
  .orderBy(desc("Total"))
  
// On prend le premier element de la colone
val bestMois = dfMois.first().getAs[Int]("Mois")
val total = dfMois.first().getAs[Double]("Total")

println(s"Le mois le plus rentable est le mois numéro $bestMois avec un chiffre d'affaires total de $total dollars.")


Le mois le plus rentable est le mois numéro 11 avec un chiffre d'affaires total de 3715537.46000095 dollars.


dfMois: Dataset[Row] = [Mois: int, Total: double]
bestMois: Int = 11
total: Double = 3715537.46000095

## Q3: Quelle ville a enregistré le plus grand nombre de ventes ?

In [9]:

// On split avec comme separateur "," et on prend le deuxieme element du tableau pour avoir la ville 
val dfCity = df.withColumn("City", split(col("Purchase Address"), ",")(1))

// Nombre de ventes par ville trier par ordre descroissant
val dfVenteByCity = dfCity
  .groupBy("City")
  .agg(count("City").alias("Total"))
  .orderBy(desc("Total"))

// On prend le premier element de la colone
val bestCity = dfVenteByCity.first().getAs[String]("City")
val total = dfVenteByCity.first().getAs[Long]("Total")

println(s"La ville qui a enregistré le plus grand nombre de ventes est $bestCity avec un total de $total ventes.")


La ville qui a enregistré le plus grand nombre de ventes est  San Francisco avec un total de 44732 ventes.


dfCity: DataFrame = [Order ID: string, Product: string ... 5 more fields]
dfVenteByCity: Dataset[Row] = [City: string, Total: bigint]
bestCity: String = " San Francisco"
total: Long = 44732L

## Q4:À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que lesclients achètent le produit ?


In [10]:

val hourDF = dfWithTimestamp.select(hour(col("Order Date")).alias("Hour"))

val bestHour = hourDF.groupBy("Hour").count().orderBy(desc("count")).first().getAs[Int]("Hour")

println(s"L'heure optimale pour diffuser des publicités est $bestHour heure.")


L'heure optimale pour diffuser des publicités est 19 heure.


hourDF: DataFrame = [Hour: int]
bestHour: Int = 19

## Q5:Quels produits sont le plus souvent vendus ensemble ?

In [11]:


val df1 = dfWithTimestamp.groupBy("Order ID")
  .agg(collect_set("Product").as("Products"))


//les paires de produits pour chaque commande
val Pairs = df1.flatMap { row =>
  val products = row.getAs[Seq[String]]("Products")
  products.combinations(2).map(pair => (pair(0), pair(1)))
}.toDF("Prod 1", "Prod 2")


// Compter les occurrences des paires de produits
val PairCounts = Pairs.groupBy("Prod 1", "Prod 2")
  .count()
  .withColumn("Occurrences", col("count").cast("Int"))
  .drop("count")


// Trier
val resultat = PairCounts.orderBy(col("Occurrences").desc)

// Affichez les 10 premiers
resultat.show(truncate=false)

+------------------------+--------------------------+-----------+
|Prod 1                  |Prod 2                    |Occurrences|
+------------------------+--------------------------+-----------+
|Lightning Charging Cable|iPhone                    |1011       |
|USB-C Charging Cable    |Google Phone              |997        |
|iPhone                  |Wired Headphones          |462        |
|Wired Headphones        |Google Phone              |422        |
|Apple Airpods Headphones|iPhone                    |373        |
|Vareebadd Phone         |USB-C Charging Cable      |368        |
|Google Phone            |Bose SoundSport Headphones|228        |
|USB-C Charging Cable    |Wired Headphones          |203        |
|Vareebadd Phone         |Wired Headphones          |149        |
|Lightning Charging Cable|Wired Headphones          |129        |
|Lightning Charging Cable|AA Batteries (4-pack)     |106        |
|USB-C Charging Cable    |Bose SoundSport Headphones|102        |
|Lightning

df1: DataFrame = [Order ID: string, Products: array<string>]
Pairs: DataFrame = [Prod 1: string, Prod 2: string]
PairCounts: DataFrame = [Prod 1: string, Prod 2: string ... 1 more field]
resultat: Dataset[Row] = [Prod 1: string, Prod 2: string ... 1 more field]

## Q6:Quel produit s'est le plus vendu?

In [12]:
// Calcul du total des quantités vendues par produit
val Vente = dfWithTimestamp.groupBy("Product")
  .agg(sum("Quantity Ordered").as("Total"))
  .orderBy(col("Total").desc)

// Récupération du produit le plus vendu (première ligne)
val topProduit = Vente.first().getAs[String]("Product")

// Affichage du produit le plus vendu
println(s"Le produit le plus vendu est : $topProduit")


Le produit le plus vendu est : AAA Batteries (4-pack)


Vente: Dataset[Row] = [Product: string, Total: double]
topProduit: String = "AAA Batteries (4-pack)"

### Les piles AAA sont des articles couramment utilisés dans de nombreux appareils électroniques en temoigne la reponse a la question 5.

In [13]:
val dfWithLabel = dfWithTimestamp.withColumn("USB-C_Ordered", when(col("Product") === "USB-C Charging Cable", 1).otherwise(0))
dfWithLabel.show()


+--------+--------------------+----------------+----------+-------------------+--------------------+-------------+
|Order ID|             Product|Quantity Ordered|Price Each|         Order Date|    Purchase Address|USB-C_Ordered|
+--------+--------------------+----------------+----------+-------------------+--------------------+-------------+
|  295665|  Macbook Pro Laptop|               1|      1700|0019-12-30 00:01:00|136 Church St, Ne...|            0|
|  295666|  LG Washing Machine|               1|     600.0|0019-12-29 07:03:00|562 2nd St, New Y...|            0|
|  295667|USB-C Charging Cable|               1|     11.95|0019-12-12 18:21:00|277 Main St, New ...|            1|
|  295668|    27in FHD Monitor|               1|    149.99|0019-12-22 15:13:00|410 6th St, San F...|            0|
|  295669|USB-C Charging Cable|               1|     11.95|0019-12-18 12:38:00|43 Hill St, Atlan...|            1|
|  295670|AA Batteries (4-p...|               1|      3.84|0019-12-31 22:58:00|2

dfWithLabel: DataFrame = [Order ID: string, Product: string ... 5 more fields]

## Creation et entrainement d'un  modèle de classification de régression logistique 

In [14]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{IntegerType, DoubleType}

val numericDF = df.withColumn("Quantity Ordered", col("Quantity Ordered").cast(IntegerType))
  .withColumn("Price Each", col("Price Each").cast(DoubleType)).na.drop()



val labeledDF = numericDF.withColumn("label", when(col("Product") === "USB-C Charging Cable", 1).otherwise(0))

val Array(trainingData, testData) = labeledDF.randomSplit(Array(0.7, 0.3))




val assembler = new VectorAssembler()
  .setInputCols(Array("Quantity Ordered", "Price Each"))
  .setOutputCol("features")

val logisticRegression = new LogisticRegression()
  .setLabelCol("label")
  .setFeaturesCol("features")

val pipeline = new Pipeline()
  .setStages(Array(assembler, logisticRegression))

val model = pipeline.fit(trainingData)


val predictions = model.transform(testData)







23/05/25 21:35:24 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/25 21:35:24 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/05/25 21:35:24 INFO LBFGS: Step Size: 16,36
23/05/25 21:35:24 INFO LBFGS: Val and Grad Norm: 0,344724 (rel: 0,0444) 0,134825
23/05/25 21:35:24 INFO LBFGS: Step Size: 1,000
23/05/25 21:35:24 INFO LBFGS: Val and Grad Norm: 0,322710 (rel: 0,0639) 0,0574524
23/05/25 21:35:24 INFO LBFGS: Step Size: 1,000
23/05/25 21:35:24 INFO LBFGS: Val and Grad Norm: 0,319214 (rel: 0,0108) 0,0188191
23/05/25 21:35:24 INFO LBFGS: Step Size: 1,000
23/05/25 21:35:24 INFO LBFGS: Val and Grad Norm: 0,312501 (rel: 0,0210) 0,0184545
23/05/25 21:35:24 INFO LBFGS: Step Size: 1,000
23/05/25 21:35:24 INFO LBFGS: Val and Grad Norm: 0,304033 (rel: 0,0271) 0,0140235
23/05/25 21:35:25 INFO LBFGS: Step Size: 1,000
23/05/25 21:35:25 INFO LBFGS: Val and Grad Norm: 0,296924 (rel: 0,0234) 0,0487951
23/05/25 21:35:2

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.classification.LogisticRegression

import org.apache.spark.ml.Pipeline

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types.{IntegerType, DoubleType}


numericDF: DataFrame = [Order ID: string, Product: string ... 4 more fields]
labeledDF: DataFrame = [Order ID: string, Product: string ... 5 more fields]
trainingData: Dataset[Row] = [Order ID: string, Product: string ... 5 more fields]
testData: Dataset[Row] = [Order ID: string, Product: string ... 5 more fields]
assembler: VectorAssembler = vecAssembler_27e11b4d3be7
logisticRegression: LogisticRegression = logreg_e283c03cce54
pipeline: Pipeline = pipeline_24ae18bfc571
model: org.apache.spark.ml.PipelineModel = pipeline_24ae18bfc571
predictions: DataFrame = [Order ID: string, Product: string ... 9 more fields]

## Q7:Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?

In [15]:


val nextClientProduct = "USB-C Charging Cable"

val nextPredictions = model.transform(numericDF.filter(col("Product") === nextClientProduct))
val probabilityVector = nextPredictions.select("probability").head().getAs[org.apache.spark.ml.linalg.DenseVector](0)
val positiveClass = probabilityVector(1)
val positiveClassFormatted = f"$positiveClass%.5f"

println(s"Probabilité de classe positive pour le prochain client qui achète un câble de chargement USB-C : $positiveClassFormatted")


Probabilité de classe positive pour le prochain client qui achète un câble de chargement USB-C : 0,21345


nextClientProduct: String = "USB-C Charging Cable"
nextPredictions: DataFrame = [Order ID: string, Product: string ... 8 more fields]
probabilityVector: org.apache.spark.ml.linalg.DenseVector = [0.7865468387155611,0.21345316128443895]
positiveClass: Double = 0.21345316128443895
positiveClassFormatted: String = "0,21345"

## Q8:Quelle est la probabilité que les prochains clients commandent un iPhone ?

In [16]:
val nextClientProduct = "iPhone"

val nextPredictions = model.transform(numericDF.filter(col("Product") === nextClientProduct))
val probabilityVector = nextPredictions.select("probability").head().getAs[org.apache.spark.ml.linalg.DenseVector](0)
val positiveClass = probabilityVector(1)
val positiveClassFormatted = f"$positiveClass%.10f"

println(s"Probabilité de classe positive pour le prochain client qui achète un Iphone : $positiveClassFormatted")

Probabilité de classe positive pour le prochain client qui achète un Iphone : 0,0000000035


nextClientProduct: String = "iPhone"
nextPredictions: DataFrame = [Order ID: string, Product: string ... 8 more fields]
probabilityVector: org.apache.spark.ml.linalg.DenseVector = [0.9999999964681876,3.5318122750305667E-9]
positiveClass: Double = 3.5318122750305667E-9
positiveClassFormatted: String = "0,0000000035"

## Q9:Quelle est la probabilité que les prochains clients commandent un téléphone Google ?

In [17]:
val nextClientProduct = "Google Phone"

val nextPredictions = model.transform(numericDF.filter(col("Product") === nextClientProduct))
val probabilityVector = nextPredictions.select("probability").head().getAs[org.apache.spark.ml.linalg.DenseVector](0)
val positiveClass = probabilityVector(1)
val positiveClassFormatted = f"$positiveClass%.10f"

println(s"Probabilité de classe positive pour le prochain client qui achète un téléphone Google : $positiveClassFormatted")

Probabilité de classe positive pour le prochain client qui achète un téléphone Google : 0,0000000494


nextClientProduct: String = "Google Phone"
nextPredictions: DataFrame = [Order ID: string, Product: string ... 8 more fields]
probabilityVector: org.apache.spark.ml.linalg.DenseVector = [0.9999999505604804,4.943951951903844E-8]
positiveClass: Double = 4.943951951903844E-8
positiveClassFormatted: String = "0,0000000494"

## Q10:Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?

In [18]:
val nextClientProduct = "Wired Headphones"

val nextPredictions = model.transform(numericDF.filter(col("Product") === nextClientProduct))
val probabilityVector = nextPredictions.select("probability").head().getAs[org.apache.spark.ml.linalg.DenseVector](0)
val positiveClass = probabilityVector(1)
val positiveClassFormatted = f"$positiveClass%.10f"

println(s"Probabilité de classe positive pour le prochain client qui achète des écouteurs filaires : $positiveClassFormatted")

Probabilité de classe positive pour le prochain client qui achète des écouteurs filaires : 0,2132759933


nextClientProduct: String = "Wired Headphones"
nextPredictions: DataFrame = [Order ID: string, Product: string ... 8 more fields]
probabilityVector: org.apache.spark.ml.linalg.DenseVector = [0.7867240066586342,0.21327599334136577]
positiveClass: Double = 0.21327599334136577
positiveClassFormatted: String = "0,2132759933"

#### Un cas d'utilisation concret de l'apprentissage automatique (Machine Learning) utilisant cet ensemble de données de ventes pourrait être la prévision la demande future de différents produits en fonction des données historiques de ventes ,pour la gestion des stocks, la planification de la production et la prise de décision stratégique en matière de vente


#### Modèle prédictif : Un modèle de régression linéaire peut être utilisé pour résoudre ce problème.

#### Étapes à suivre :

#### Préparation des données : Nettoyez les données en éliminant les valeurs manquantes, en normalisant si nécessaire, et en préparant les caractéristiques pertinentes pour l'entraînement du modèle.

#### Séparation des données : Divisez l'ensemble de données en ensembles d'entraînement et de test. Utilisez les données historiques de ventes pour l'entraînement et réservez une partie des données pour évaluer les performances du modèle.

#### Création du modèle : Choisissez un modèle approprié pour la prévision de la demande, tel qu'un modèle de régression linéaire. Configurez les hyperparamètres du modèle et entraînez-le sur l'ensemble d'entraînement.

#### Évaluation du modèle : Utilisez l'ensemble de test pour évaluer les performances du modèle en comparant les valeurs prédites de la demande avec les valeurs réelles. Utilisez des mesures d'évaluation telles que l'erreur quadratique moyenne (RMSE) ou le coefficient de détermination (R2).

#### Prévision de la demande future : Une fois le modèle entraîné et évalué, utilisez-le pour faire des prédictions sur la demande future des produits

